In [85]:
import numpy as np
import math
import cv2
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt
#%matplotlib inline

def cal_chessBoard(images, nx, ny):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:ny,0:nx].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images


    # Step through the list and search for chessboard corners
    for fname in images:
        #print("test")
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (ny,nx),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (ny,nx), corners, ret)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)

    #cv2.destroyAllWindows()
    return objpoints, imgpoints

def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist

def corners_unwarp(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

def edge_detection(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    rgb_r_channel = img[:,:,0]
    #rgb_g_channel = image[:,:,1]
    #rgb_b_channel = image[:,:,2]

    rgb_thresh_min = 220
    rgb_thresh_max = 255
    rgb_r_binary = np.zeros_like(rgb_r_channel)
    rgb_r_binary[(rgb_r_channel >= rgb_thresh_min) & (rgb_r_channel <= rgb_thresh_max)] = 1
    
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab_l_channel = lab[:,:,0]
    lab_b_channel = lab[:,:,2]

    lab_l_thresh_min = 220
    lab_l_thresh_max = 255
    lab_l_binary = np.zeros_like(lab_l_channel)
    lab_l_binary[(lab_l_channel >= lab_l_thresh_min) & (lab_l_channel <= lab_l_thresh_max)] = 1
    
    lab_b_thresh_min = 170
    lab_b_thresh_max = 255
    lab_b_binary = np.zeros_like(lab_b_channel)
    lab_b_binary[(lab_b_channel >= lab_b_thresh_min) & (lab_b_channel <= lab_b_thresh_max)] = 1
    
    #hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    #s_channel = hls[:,:,2]
    #hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #v_channel = hsv[:,:,2]

    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    #sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    #abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    #scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    #thresh_min = 40
    #thresh_max = 90
    #sxbinary = np.zeros_like(scaled_sobel)
    #sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    #plt.imshow(sxbinary)
    # Threshold color channel
    #s_thresh_min = 190
    #s_thresh_max = 255
    #s_binary = np.zeros_like(s_channel)
    #s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    #v_thresh_min = 235
    #v_thresh_max = 255
    #v_binary = np.zeros_like(v_channel)
    #v_binary[(v_channel >= v_thresh_min) & (v_channel <= v_thresh_max)] = 1
    
    #l_thresh_min = 215
    #l_thresh_max = 255
    #l_binary = np.zeros_like(l_channel)
    #l_binary[(l_channel >= l_thresh_min) & (l_channel <= l_thresh_max)] = 1
    #plt.imshow(s_binary)
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(rgb_r_channel)
    combined_binary[(rgb_r_binary == 1) | (lab_l_binary == 1) | (lab_b_binary == 1)] = 1
    #combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

def find_lane_pixels(binary_warped,left_curve,right_curve):# sliding window method:
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])# the max value index of left side
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint# the max value index of left side

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    left_offset=0
    right_offset=0
    if left_curve>0:
        left_offset=20
    elif left_curve<0:
        left_offset=-20
    
    if right_curve>0:
        right_offset=20
    elif right_curve<0:
        right_offset=-20
        

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    #print("nonzero: ", nonzero)
    nonzeroy = np.array(nonzero[0])
    #print("nonzeroy: ", nonzeroy)
    nonzerox = np.array(nonzero[1])
    #print("nonzerox: ", nonzerox)
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Find the four below boundaries of the window
        win_xleft_low = leftx_current - margin + left_offset
        win_xleft_high = leftx_current + margin + left_offset
        win_xright_low = rightx_current - margin + right_offset
        win_xright_high = rightx_current + margin + right_offset
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        #(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),
        #(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty


def fit_polynomial(binary_warped, left_curve=0, right_curve=0):
    # Find our lane pixels first
    leftx, lefty, rightx, righty = find_lane_pixels(binary_warped,left_curve,right_curve)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    
    #plt.plot(left_fitx, ploty, color='blue')
    #plt.plot(right_fitx, ploty, color='blue')

    return left_fit, right_fit

def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    #print("left_fit: ",left_fit)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Calc both polynomials using ploty, left_fit and right_fit
    left_fitx = left_fit[0]*ploty**2+left_fit[1]*ploty+left_fit[2]
    right_fitx = right_fit[0]*ploty**2+right_fit[1]*ploty+right_fit[2]
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped, left_fit, right_fit, curve_change_rate=0):

    margin = 70
    offset = 0
    #print("binary_warped.shape[0]: ",binary_warped.shape[0])
    if curve_change_rate<-0.003 or curve_change_rate>0.003:#radius decreased
        margin = 50
        offset = 70
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    ratio = (binary_warped.shape[0]-nonzeroy)/binary_warped.shape[0]
    optimized_margin = margin + offset*ratio
    # Use previous polynomials
    left_lane_inds = (nonzerox>left_fit[0]*nonzeroy**2+left_fit[1]*nonzeroy+left_fit[2]-optimized_margin)&(nonzerox<left_fit[0]*nonzeroy**2+left_fit[1]*nonzeroy+left_fit[2]+optimized_margin)
    right_lane_inds = (nonzerox>right_fit[0]*nonzeroy**2+right_fit[1]*nonzeroy+right_fit[2]-optimized_margin)&(nonzerox<right_fit[0]*nonzeroy**2+right_fit[1]*nonzeroy+right_fit[2]+optimized_margin)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    return ploty, left_fitx, right_fitx, left_fit, right_fit

def measure_curvature_real(ploty, left_fit, right_fit, mx, my):
    y_eval = np.max(ploty)
    a_left=left_fit[0]*mx/(my**2)
    b_left=left_fit[1]*mx/my
    a_right=right_fit[0]*mx/(my**2)
    b_right=right_fit[1]*mx/my
    
    left_curverad = (1+(2*a_left*y_eval+b_left)**2)**1.5/abs(2*a_left)  ## Implement the calculation of the left line here
    right_curverad = (1+(2*a_right*y_eval+b_right)**2)**1.5/abs(2*a_right)  ## Implement the calculation of the right line here
    
    return left_curverad, right_curverad

def draw_lanes(warped, undist, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    left_lane_plot = (np.asarray([left_fitx, ploty]).T).astype(np.int32)
    right_lane_plot = (np.asarray([right_fitx, ploty]).T).astype(np.int32)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    cv2.polylines(color_warp,[left_lane_plot],False,(255,0,0),20)
    cv2.polylines(color_warp,[right_lane_plot],False,(255,0,0),20)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        self.recent_fit = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = []  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None
        self.recent_radius_of_curvature = []
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        self.recent_base_pos = []
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        self.recent_diffs=[]
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 
        self.space = None
        self.recent_curve_change_rate=[]


#plt.imshow(image)
#print("image.shape: ",image.shape)
images = glob.glob('camera_cal/calibration*.jpg')
objpoints, imgpoints = cal_chessBoard(images, 6, 9)

src = np.float32([[560,470],[720,470],[0,720],[1280,720]])
dst = np.float32([[250,0],[1030,0],[140,720],[1140,720]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

mx= 3.7/700# meters per pixel
my=30/720# meters per pixel

#print("1",objpoints, imgpoints)
#print("2",mtx, dist)
#print("3",M)
#print("4",Minv)
print("Done!")

Done!


In [88]:
image = plt.imread('test_images/test4.jpg')
mtx, dist = cal_undistort(image, objpoints, imgpoints)
left_lane=Line()
right_lane=Line()
fail_times=0
current_frame=0
lane_space=[]

def process_image(image):
    global left_lane, right_lane, fail_times,current_frame
    #print("current_frame: ",current_frame)
    current_frame+=1
    #resize image to 1280*720(X*Y)
    if image.shape[1]!=1280 or image.shape[0]!=720:
        resized_img = cv2.resize(image, (1280, 720))
    else:
        resized_img = image
    #undisortion
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    #undist = undist[:,:,[2,1,0]]
    #plt.imshow(undist)
    #edge detection
    edge = edge_detection(undist)
    #plt.imshow(edge)
    #perspective transform
    warped = corners_unwarp(edge, src, dst)
    #plt.imshow(warped)
    if len(left_lane.best_fit)!=0 and len(right_lane.best_fit)!=0:# if lane line best_fit exist
        #print("position A")
        try:
            left_delta_angle = 2*math.asin(30/left_lane.radius_of_curvature) - 2*math.asin(30/left_lane.recent_radius_of_curvature[-2])
            right_delta_angle = 2*math.asin(30/right_lane.radius_of_curvature) - 2*math.asin(30/right_lane.recent_radius_of_curvature[-2])
            curve_change_rate = (left_delta_angle + right_delta_angle)/2
            left_lane.recent_curve_change_rate.append(curve_change_rate)
            left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_lane.best_fit, right_lane.best_fit, curve_change_rate)
            #update .detected
            #print("position A.1")

            left_lane.detected = True
            right_lane.detected = True
        except:
            #print("position A.2")
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
        if fail_times>=5:# to use sliding window method:
            #print("position A.3")
            try:
                left_fit, right_fit = fit_polynomial(warped, left_lane.best_fit[0], right_lane.best_fit[0])# to use sliding window method:
                left_lane.best_fit = []
                right_lane.best_fit = []
                left_lane.recent_fit = []
                right_lane.recent_fit = []
                left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_fit, right_fit)
                #update .detected
                #print("position A.4")

                left_lane.detected = True
                right_lane.detected = True
            except:
                #print("position A.5")
                #print("max fail_times reached, but lanes still not detected")
                return None
    else:# if lane line best_fit NOT exist:
        #print("position B")
        try:
            left_fit, right_fit = fit_polynomial(warped)# to use sliding window method:
            left_lane.best_fit = []
            right_lane.best_fit = []
            left_lane.recent_fit = []
            right_lane.recent_fit = []
            left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_fit, right_fit)
            #update .detected
            #print("position B.1")

            left_lane.detected = True
            right_lane.detected = True
        except:
            #update .detected
            #print("position B.2")
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
            if fail_times>=5:#if fail from the beginning and reached max fail_times
                #print("position B.3")
                #print("max fail_times reached, but lanes still not detected")
                return None
    
    if left_lane.detected and right_lane.detected:# if lane line detected in latest iteration
        #print("position C")
        if len(left_lane.best_fit)!=0 and len(right_lane.best_fit)!=0:
            #left_top_x=left_lane.allx[0]
            #right_top_x=right_lane.allx[0]
            #left_bottom_X=left_lane.allx[-1]
            #right_bottom_x=right_lane.allx[-1]
            #left_lane.current_radius_of_curvature, right_lane.current_radius_of_curvature = measure_curvature_real(left_lane.ally, left_lane.current_fit, right_lane.current_fit, mx, my)
            
            left_lane.diffs = left_lane.current_fit - left_lane.best_fit
            right_lane.diffs = right_lane.current_fit - right_lane.best_fit
            left_lane.recent_diffs.append(abs(left_lane.diffs[0]))
            right_lane.recent_diffs.append(abs(right_lane.diffs[0]))
            left_lane.space = np.max(right_lane.allx - left_lane.allx)-np.min(right_lane.allx - left_lane.allx)
            lane_space.append(left_lane.space)# for debugging measurements
        else:
            left_lane.diffs = np.array([0,0,0], dtype='float')
            right_lane.diffs = np.array([0,0,0], dtype='float')
            left_lane.space = 0.0
        #print("left_lane.diffs: ",left_lane.diffs)
        #print("right_lane.diffs: ",right_lane.diffs)
        #print("left_lane.parallel: ",left_lane.parallel)

        #sanity check defluctuation(remove sudden change and inparallel):
        if abs(left_lane.diffs[0])<2e-4 and abs(right_lane.diffs[0])<2e-4 and left_lane.space<100:# to filter out strange fits
            #print(abs(left_lane.diffs[0]))
            #print(abs(right_lane.diffs[0]))
            #print("position C.1")
            #then to get/update best_fit:
            left_lane.recent_fit.append(left_lane.current_fit)
            right_lane.recent_fit.append(right_lane.current_fit)
            
            # to get average:
            if len(left_lane.recent_fit)==1:
                #print("position C.2")
                left_lane.best_fit=left_lane.current_fit
                #print("len(left_lane.best_fit): ",len(left_lane.best_fit))
                #print("left_lane.best_fit: ",left_lane.best_fit)
                right_lane.best_fit=right_lane.current_fit
                #print("len(right_lane.best_fit): ",len(right_lane.best_fit))
                #print("right_lane.best_fit: ",right_lane.best_fit)
            else:
                #print("position C.3")
                if len(left_lane.recent_fit)==4:
                    #print("position C.4")
                    left_lane.recent_fit.pop(0)
                    right_lane.recent_fit.pop(0)
                left_lane.best_fit=np.mean(left_lane.recent_fit,0)
                right_lane.best_fit=np.mean(right_lane.recent_fit,0)
            # only to define bestx AFTER filtering, otherwise to keep using former bestx
            left_lane.bestx = left_lane.allx
            right_lane.bestx = right_lane.allx
            fail_times=0 #reset
        else:
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
        #print("position D")
        
        left_lane.radius_of_curvature, right_lane.radius_of_curvature = measure_curvature_real(left_lane.ally, left_lane.best_fit, right_lane.best_fit, mx, my)
        left_lane.radius_of_curvature=round(left_lane.radius_of_curvature,2)
        right_lane.radius_of_curvature=round(right_lane.radius_of_curvature,2)
        
        left_lane.line_base_pos = ((left_lane.bestx[-1]+right_lane.bestx[-1])//2 - image.shape[1]//2)/(right_lane.bestx[-1]-left_lane.bestx[-1])*3
        left_lane.line_base_pos=round(left_lane.line_base_pos,2)
        #print("left_lane.radius_of_curvature: ",left_lane.radius_of_curvature)
        #print("right_lane.radius_of_curvature: ",right_lane.radius_of_curvature)
        left_lane.recent_radius_of_curvature.append(left_lane.radius_of_curvature)
        right_lane.recent_radius_of_curvature.append(right_lane.radius_of_curvature)
        left_lane.recent_base_pos.append(left_lane.line_base_pos)
    # if sanity check failed, then use last time lane lines. but if max fail_times reached, then use silding window method
    #print("left_lane.bestx",left_lane.bestx)
    #print("right_lane.bestx",right_lane.bestx)
    image_output = draw_lanes(warped, image, left_lane.bestx, right_lane.bestx, left_lane.ally, Minv)
    #print("position E")
    left_lane.detected = False# back to default False for next loop frame
    right_lane.detected = False
    vehicle_data_1 = "Left lane Curve: "+str(left_lane.radius_of_curvature)+" meter"
    vehicle_data_2 = "Right lane Curve: "+str(right_lane.radius_of_curvature)+" meter"
    vehicle_data_3 = "Center Offset: "+str(left_lane.line_base_pos)+" meter"
    #vehicle_data_4 = "Left diffs: "+str(abs(left_lane.diffs[0]))
    #vehicle_data_5 = "Right diffs: "+str(abs(right_lane.diffs[0]))
    #vehicle_data_6 = "space_dev: "+str(left_lane.space)
    
    cv2.putText(image_output, vehicle_data_1, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    cv2.putText(image_output, vehicle_data_2, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    cv2.putText(image_output, vehicle_data_3, (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    #cv2.putText(image_output, vehicle_data_4, (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    #cv2.putText(image_output, vehicle_data_5, (50,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    #cv2.putText(image_output, vehicle_data_6, (50,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 3)
    return image_output

#image_output = process_image(image)
#image_output = image_output[:,:,[2,1,0]]
#plt.imshow(image_output)

In [89]:
white_output = 'project_video_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
#print("left_lane.recent_curve_change_rate",left_lane.recent_curve_change_rate)
#for item in lane_space:
#    print(item)
#for item in left_lane.recent_radius_of_curvature:
#    print(item)
#print("curvatures and vehicle position offset are as below:")
#print("#### left curvatures for each frame: ",left_lane.recent_radius_of_curvature)
#print("#### right curvatures for each frame: ",right_lane.recent_radius_of_curvature)
#print("#### vehicle position offset for each frame: ",left_lane.recent_base_pos.append)

t:   0%|          | 3/1260 [00:00<00:54, 22.95it/s, now=None]

Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4



Moviepy - Done !
Moviepy - video ready project_video_out.mp4
CPU times: user 3min 20s, sys: 18.1 s, total: 3min 38s
Wall time: 1min 58s


In [12]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [5]:
test1=[1,2,3,4,5]
print(test1[-2])

4


In [16]:
print(left_lane.recent_diffs)

[4.450096168832175e-05, 1.494026503320422e-06, 1.1306025862969986e-05, 3.008812817080958e-06, 2.2435119893015584e-05, 8.083710002423916e-07, 2.6086667465472657e-05, 4.763567849146958e-07, 1.1806891176220738e-05, 2.2446872021213263e-05, 3.879735115466558e-05, 1.4292145822094313e-05, 3.275163322659236e-05, 3.9697610268119184e-05, 3.431037385539544e-06, 4.605972655574221e-06, 3.6376108768849984e-06, 5.757913231195787e-05, 2.789763465007401e-05, 1.680639949853606e-05, 2.240654939377598e-05, 2.800688971989518e-05, 1.874855826284988e-05, 8.89777963632141e-06, 9.94046677148445e-06, 4.36718247026133e-05, 2.6952143882928923e-05, 5.437042337137684e-07, 2.7239458410315676e-05, 2.7622762469317e-06, 5.5352434995039875e-06, 1.3668662113410505e-05, 3.0455274806873226e-05, 3.597156395008383e-05, 9.092276345490667e-06, 1.283230853853912e-05, 1.9500683955267842e-05, 1.2456570606887387e-05, 2.083881283231775e-07, 2.7177552652839183e-05, 2.2692942453484305e-05, 5.518070481325214e-06, 3.693868030380055e-05

In [19]:
print(0>1e-6)

False
